In [1]:

%pip install snowflake-connector-python
%pip install snowflake-snowpark-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:

import os
import configparser
from snowflake.snowpark import Session
import snowflake.snowpark.functions as f
from snowflake.snowpark.window import Window
from datetime import date


In [3]:
config = configparser.ConfigParser()
config.read('config.ini')


['config.ini']

### creating snowpark session

In [4]:

def snowpark_session_create():
    connection_params = {
        "account": config.get("snowflake", "account"),
        "user": config.get("snowflake", "user"),
        "password": config.get("snowflake", "password"),
        "role": config.get("snowflake", "role"),
        "warehouse": config.get("snowflake", "warehouse"),
        "database": config.get("snowflake", "database"),
        "schema": config.get("snowflake", "schema")
    }

    session = Session.builder.configs(connection_params).create()
    return session


In [6]:
demo_session = snowpark_session_create()

In [7]:
df = demo_session.sql("SELECT * FROM DEMO.RAW.STG_CITIES LIMIT ")
df.show()


-----------------------------------------------------------------------------------------------------------------------------
|"CITY_ID"  |"CITY"  |"DATE"      |"TAVG"  |"TMIN"  |"TMAX"  |"PRCP"  |"SNOW"  |"WDIR"  |"WSPD"  |"WPGT"  |"PRES"  |"TSUN"  |
-----------------------------------------------------------------------------------------------------------------------------
|1          |Kyiv    |2022-08-29  |25.3    |16.6    |31.5    |0.0     |NULL    |278.0   |6.8     |27.8    |1012.5  |NULL    |
|1          |Kyiv    |2022-09-09  |13.1    |6.8     |18.3    |0.3     |NULL    |82.0    |7.0     |24.1    |1022.4  |NULL    |
|1          |Kyiv    |2022-09-08  |14.9    |9.2     |20.5    |0.0     |NULL    |15.0    |4.7     |22.2    |1019.4  |NULL    |
|1          |Kyiv    |2022-09-07  |12.4    |6.7     |19.0    |0.0     |NULL    |281.0   |4.4     |22.2    |1022.3  |NULL    |
|1          |Kyiv    |2022-09-06  |10.9    |6.8     |14.6    |0.0     |NULL    |344.0   |5.7     |25.9    |1024.7  |NU

In [7]:
for row in df.collect():
    print(row)

Row(CITY_ID=1, CITY='Kyiv', DATE=datetime.date(2022, 8, 29), TAVG=25.3, TMIN=16.6, TMAX=31.5, PRCP=0.0, SNOW=None, WDIR=278.0, WSPD=6.8, WPGT=27.8, PRES=1012.5, TSUN=None)
Row(CITY_ID=1, CITY='Kyiv', DATE=datetime.date(2022, 9, 9), TAVG=13.1, TMIN=6.8, TMAX=18.3, PRCP=0.3, SNOW=None, WDIR=82.0, WSPD=7.0, WPGT=24.1, PRES=1022.4, TSUN=None)
Row(CITY_ID=1, CITY='Kyiv', DATE=datetime.date(2022, 9, 8), TAVG=14.9, TMIN=9.2, TMAX=20.5, PRCP=0.0, SNOW=None, WDIR=15.0, WSPD=4.7, WPGT=22.2, PRES=1019.4, TSUN=None)
Row(CITY_ID=1, CITY='Kyiv', DATE=datetime.date(2022, 9, 7), TAVG=12.4, TMIN=6.7, TMAX=19.0, PRCP=0.0, SNOW=None, WDIR=281.0, WSPD=4.4, WPGT=22.2, PRES=1022.3, TSUN=None)
Row(CITY_ID=1, CITY='Kyiv', DATE=datetime.date(2022, 9, 6), TAVG=10.9, TMIN=6.8, TMAX=14.6, PRCP=0.0, SNOW=None, WDIR=344.0, WSPD=5.7, WPGT=25.9, PRES=1024.7, TSUN=None)
Row(CITY_ID=1, CITY='Kyiv', DATE=datetime.date(2022, 9, 5), TAVG=13.9, TMIN=12.6, TMAX=18.4, PRCP=0.7, SNOW=None, WDIR=343.0, WSPD=5.8, WPGT=25.9, PRE

### point session to the snowflake demo db,schema,table


In [8]:
demo_session.use_database("demo")
demo_session.use_schema("raw")
demo_session.table("stg_cities")

In [9]:
df = df.withColumn("DATE", f.to_date(f.col("DATE")))

In [10]:
print(df.count())
df.show()


7305
-----------------------------------------------------------------------------------------------------------------------------
|"CITY_ID"  |"CITY"  |"TAVG"  |"TMIN"  |"TMAX"  |"PRCP"  |"SNOW"  |"WDIR"  |"WSPD"  |"WPGT"  |"PRES"  |"TSUN"  |"DATE"      |
-----------------------------------------------------------------------------------------------------------------------------
|1          |Kyiv    |25.3    |16.6    |31.5    |0.0     |NULL    |278.0   |6.8     |27.8    |1012.5  |NULL    |2022-08-29  |
|1          |Kyiv    |13.1    |6.8     |18.3    |0.3     |NULL    |82.0    |7.0     |24.1    |1022.4  |NULL    |2022-09-09  |
|1          |Kyiv    |14.9    |9.2     |20.5    |0.0     |NULL    |15.0    |4.7     |22.2    |1019.4  |NULL    |2022-09-08  |
|1          |Kyiv    |12.4    |6.7     |19.0    |0.0     |NULL    |281.0   |4.4     |22.2    |1022.3  |NULL    |2022-09-07  |
|1          |Kyiv    |10.9    |6.8     |14.6    |0.0     |NULL    |344.0   |5.7     |25.9    |1024.7  |NULL    |2

In [11]:
df = (
    df.select(
        "CITY",
        "DATE",
        f.col("TAVG").alias("TEMP_AVG"),
        f.col("TMAX").alias("TEMP_MAX"),
        f.col("TMIN").alias("TEMP_MIN"),
        f.col("PRCP").alias("TOTAL_PRECIPITATION"),
        f.col("WSPD").alias("AVG_WIND_SPEED"),
        f.col("WPGT").alias("WIND_PEAK_GUST"),
        f.col("PRES").alias("SEA_LEVEL_AIR_PRESSURE")
    )
    .filter(f.col("DATE").between(date(2020, 6, 1), date(2020, 8, 31)))
    .sort(f.col("DATE").asc())
)




In [12]:
df.show(100)

-------------------------------------------------------------------------------------------------------------------------------------------------------
|"CITY"  |"DATE"      |"TEMP_AVG"  |"TEMP_MAX"  |"TEMP_MIN"  |"TOTAL_PRECIPITATION"  |"AVG_WIND_SPEED"  |"WIND_PEAK_GUST"  |"SEA_LEVEL_AIR_PRESSURE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|Oslo    |2020-06-01  |19.6        |27.2        |10.3        |0.1                    |9.9               |23.0              |1027.3                    |
|Zagreb  |2020-06-01  |18.4        |23.2        |11.8        |0.0                    |8.3               |22.2              |1016.9                    |
|Kyiv    |2020-06-01  |11.8        |16.0        |9.0         |NULL                   |16.8              |35.2              |1007.8                    |
|Munich  |2020-06-01  |16.1        |22.5        |8.3         |0.0                    |10

In [13]:
df = df.na.fill({"TOTAL_PRECIPITATION": 0, "AVG_WIND_SPEED": 0, "WIND_PEAK_GUST": 0, "SEA_LEVEL_AIR_PRESSURE": 0})
df.show(100)


-------------------------------------------------------------------------------------------------------------------------------------------------------
|"CITY"  |"DATE"      |"TEMP_AVG"  |"TEMP_MAX"  |"TEMP_MIN"  |"TOTAL_PRECIPITATION"  |"AVG_WIND_SPEED"  |"WIND_PEAK_GUST"  |"SEA_LEVEL_AIR_PRESSURE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|Oslo    |2020-06-01  |19.6        |27.2        |10.3        |0.1                    |9.9               |23.0              |1027.3                    |
|Zagreb  |2020-06-01  |18.4        |23.2        |11.8        |0.0                    |8.3               |22.2              |1016.9                    |
|Kyiv    |2020-06-01  |11.8        |16.0        |9.0         |0.0                    |16.8              |35.2              |1007.8                    |
|Munich  |2020-06-01  |16.1        |22.5        |8.3         |0.0                    |10

In [14]:
df_count2020 = df.withColumn("MONTH", f.monthname(f.col("DATE")))\
    .withColumn("YEAR",f.year(f.col("DATE"))) \
    .group_by("CITY", "MONTH","YEAR") \
    .agg(f.avg("TEMP_AVG").alias("MONTHLY_AVG_TEMP"),
         f.max("TEMP_MAX").alias("WARMEST_SUMMER_MONTH"),
         f.min("TEMP_MIN").alias("COLDEST_SUMMER_MONTH"))\
    .sort(f.col("CITY"),f.col("MONTH").desc())

In [15]:
df_count2020.show(100)

----------------------------------------------------------------------------------------------------
|"CITY"  |"MONTH"  |"YEAR"  |"MONTHLY_AVG_TEMP"  |"WARMEST_SUMMER_MONTH"  |"COLDEST_SUMMER_MONTH"  |
----------------------------------------------------------------------------------------------------
|Kyiv    |Jun      |2020    |21.64               |33.3                    |5.0                     |
|Kyiv    |Jul      |2020    |21.451612903225808  |32.7                    |9.0                     |
|Kyiv    |Aug      |2020    |20.758064516129032  |34.1                    |7.0                     |
|Munich  |Jun      |2020    |16.81               |29.2                    |8.3                     |
|Munich  |Jul      |2020    |20.0                |32.7                    |10.6                    |
|Munich  |Aug      |2020    |20.212903225806453  |34.2                    |10.0                    |
|Oslo    |Jun      |2020    |18.796666666666667  |30.9                    |8.6             

In [16]:
df2 = demo_session.table("stg_cities")
df2 = (
    df2.select(
        "CITY",
        "DATE",
        f.col("TAVG").alias("TEMP_AVG"),
        f.col("TMAX").alias("TEMP_MAX"),
        f.col("TMIN").alias("TEMP_MIN"),
        f.col("PRCP").alias("TOTAL_PRECIPITATION"),
        f.col("WSPD").alias("AVG_WIND_SPEED"),
        f.col("WPGT").alias("WIND_PEAK_GUST"),
        f.col("PRES").alias("SEA_LEVEL_AIR_PRESSURE")
    )
    .filter(f.col("DATE").between(date(2023, 6, 1), date(2023, 8, 31)))
    .sort(f.col("DATE").asc())
)
df = df.na.fill({"TOTAL_PRECIPITATION": 0, "AVG_WIND_SPEED": 0, "WIND_PEAK_GUST": 0, "SEA_LEVEL_AIR_PRESSURE": 0})
df.show(100)




-------------------------------------------------------------------------------------------------------------------------------------------------------
|"CITY"  |"DATE"      |"TEMP_AVG"  |"TEMP_MAX"  |"TEMP_MIN"  |"TOTAL_PRECIPITATION"  |"AVG_WIND_SPEED"  |"WIND_PEAK_GUST"  |"SEA_LEVEL_AIR_PRESSURE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|Zagreb  |2020-06-01  |18.4        |23.2        |11.8        |0.0                    |8.3               |22.2              |1016.9                    |
|Oslo    |2020-06-01  |19.6        |27.2        |10.3        |0.1                    |9.9               |23.0              |1027.3                    |
|Munich  |2020-06-01  |16.1        |22.5        |8.3         |0.0                    |10.4              |34.2              |1019.2                    |
|Kyiv    |2020-06-01  |11.8        |16.0        |9.0         |0.0                    |16

In [17]:
df_count2023 = df2.withColumn("MONTH", f.monthname(f.col("DATE")))\
    .withColumn("YEAR",f.year(f.col("DATE"))) \
    .group_by("CITY", "MONTH","YEAR") \
    .agg(f.avg("TEMP_AVG").alias("MONTHLY_AVG_TEMP"),
         f.max("TEMP_MAX").alias("WARMEST_SUMMER_MONTH"),
         f.min("TEMP_MIN").alias("COLDEST_SUMMER_MONTH"))\
    .sort(f.col("CITY"),f.col("MONTH").desc())     

In [18]:
df_count2023.show(100)

----------------------------------------------------------------------------------------------------
|"CITY"  |"MONTH"  |"YEAR"  |"MONTHLY_AVG_TEMP"  |"WARMEST_SUMMER_MONTH"  |"COLDEST_SUMMER_MONTH"  |
----------------------------------------------------------------------------------------------------
|Kyiv    |Jun      |2023    |19.720000000000002  |31.1                    |7.9                     |
|Kyiv    |Jul      |2023    |21.306451612903224  |32.7                    |13.5                    |
|Kyiv    |Aug      |2023    |23.758064516129032  |35.9                    |13.6                    |
|Munich  |Jun      |2023    |19.970000000000002  |34.5                    |9.6                     |
|Munich  |Jul      |2023    |20.95483870967742   |35.9                    |10.4                    |
|Munich  |Aug      |2023    |20.070967741935487  |34.5                    |10.0                    |
|Oslo    |Jun      |2023    |18.85666666666667   |31.8                    |4.6             

### join tables for comparison

In [19]:

joined_df = df_count2020.join(
    df_count2023, 
    (df_count2020.CITY == df_count2023.CITY) & 
    (df_count2020.MONTH == df_count2023.MONTH) 
).select(
    df_count2020.CITY.alias("CITY_2020"),
    df_count2020.MONTH.alias("MONTH_2020"),
    df_count2020.WARMEST_SUMMER_MONTH.alias("WARMEST_SUMMER_MONTH_2020"),
    df_count2023.CITY.alias("CITY_2023"),
    df_count2023.MONTH.alias("MONTH_2023"),
    df_count2023.WARMEST_SUMMER_MONTH.alias("WARMEST_SUMMER_MONTH_2023")
).sort(f.col("CITY_2020"),f.col("MONTH_2020").desc())



In [20]:
joined_df.show(100)

-----------------------------------------------------------------------------------------------------------------------
|"CITY_2020"  |"MONTH_2020"  |"WARMEST_SUMMER_MONTH_2020"  |"CITY_2023"  |"MONTH_2023"  |"WARMEST_SUMMER_MONTH_2023"  |
-----------------------------------------------------------------------------------------------------------------------
|Kyiv         |Jun           |33.3                         |Kyiv         |Jun           |31.1                         |
|Kyiv         |Jul           |32.7                         |Kyiv         |Jul           |32.7                         |
|Kyiv         |Aug           |34.1                         |Kyiv         |Aug           |35.9                         |
|Munich       |Jun           |29.2                         |Munich       |Jun           |34.5                         |
|Munich       |Jul           |32.7                         |Munich       |Jul           |35.9                         |
|Munich       |Aug           |34.2      

In [21]:
df_count2020 = df.withColumn("SUMMER_MAX_TEMP",f.col("TEMP_MAX")) \
    .group_by("CITY") \
    .agg(f.max("TEMP_MAX").alias("HIGHEST_SUMMER_TEMP_2020"))\
    .sort(f.col("CITY"))

   

In [22]:
df_count2020.show(100)

---------------------------------------
|"CITY"  |"HIGHEST_SUMMER_TEMP_2020"  |
---------------------------------------
|Kyiv    |34.1                        |
|Munich  |34.2                        |
|Oslo    |30.9                        |
|Paris   |39.3                        |
|Zagreb  |34.0                        |
---------------------------------------



In [23]:
df_count2023 = df2.withColumn("SUMMER_MAX_TEMP",f.col("TEMP_MAX")) \
    .group_by("CITY") \
    .agg(f.max("TEMP_MAX").alias("HIGHEST_SUMMER_TEMP_2023"))\
    .sort(f.col("CITY"))

   

In [24]:
df_count2023.show()

---------------------------------------
|"CITY"  |"HIGHEST_SUMMER_TEMP_2023"  |
---------------------------------------
|Kyiv    |35.9                        |
|Munich  |35.9                        |
|Oslo    |31.8                        |
|Paris   |33.6                        |
|Zagreb  |35.7                        |
---------------------------------------



In [25]:
join_highest_summer = df_count2020.join(
    df_count2023, 
    (df_count2020.CITY == df_count2023.CITY) 
).select(
    df_count2020.CITY.alias("CITY_2020"),
    df_count2020.HIGHEST_SUMMER_TEMP_2020.alias("HIGHEST_SUMMER_TEMP_2020"),
    df_count2023.CITY.alias("CITY_2023"),
    df_count2023.HIGHEST_SUMMER_TEMP_2023.alias("HIGHEST_SUMMER_TEMP_2023")
).sort(f.col("CITY_2020"))




In [26]:

join_highest_summer.show(100)

---------------------------------------------------------------------------------------
|"CITY_2020"  |"HIGHEST_SUMMER_TEMP_2020"  |"CITY_2023"  |"HIGHEST_SUMMER_TEMP_2023"  |
---------------------------------------------------------------------------------------
|Kyiv         |34.1                        |Kyiv         |35.9                        |
|Munich       |34.2                        |Munich       |35.9                        |
|Oslo         |30.9                        |Oslo         |31.8                        |
|Paris        |39.3                        |Paris        |33.6                        |
|Zagreb       |34.0                        |Zagreb       |35.7                        |
---------------------------------------------------------------------------------------



### functions

In [27]:
# a column that puts two totals into an array
#lefr a comment
join_highest_summer = join_highest_summer.withColumn(
    "2020vs2023_highest_summer_temp",
    f.array_construct(
        f.col("HIGHEST_SUMMER_TEMP_2020").cast("int"),
        f.col("HIGHEST_SUMMER_TEMP_2023").cast("int")
    )
).withColumn(
    "COMMENT",
    f.concat(f.lit("Code was created by "), f.current_user())
)


In [28]:
join_highest_summer.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------
|"CITY_2020"  |"HIGHEST_SUMMER_TEMP_2020"  |"CITY_2023"  |"HIGHEST_SUMMER_TEMP_2023"  |"2020vs2023_highest_summer_temp"  |"COMMENT"                 |
-----------------------------------------------------------------------------------------------------------------------------------------------------
|Kyiv         |34.1                        |Kyiv         |35.9                        |[                                 |Code was created by OLHA  |
|             |                            |             |                            |  34,                             |                          |
|             |                            |             |                            |  36                              |                          |
|             |                            |             |                            |]            

In [29]:
join_highest_summer.drop(f.col("HIGHEST_SUMMER_TEMP_2020"),f.col("HIGHEST_SUMMER_TEMP_2023")).show()

-------------------------------------------------------------------------------------------
|"CITY_2020"  |"CITY_2023"  |"2020vs2023_highest_summer_temp"  |"COMMENT"                 |
-------------------------------------------------------------------------------------------
|Kyiv         |Kyiv         |[                                 |Code was created by OLHA  |
|             |             |  34,                             |                          |
|             |             |  36                              |                          |
|             |             |]                                 |                          |
|Munich       |Munich       |[                                 |Code was created by OLHA  |
|             |             |  34,                             |                          |
|             |             |  36                              |                          |
|             |             |]                                 |                

### close the session

In [ ]:
demo_session.close()